<a href="https://colab.research.google.com/github/eshita1802/Continuous-Authentication/blob/main/NBERT%20final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch transformers datasets scikit-learn numpy pandas tqdm matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 915.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive. mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Define a dataset class
class SensorDataset(Dataset):
    def __init__(self, file_paths):
        self.data = []
        self.labels = []

        for file_path in file_paths:
            df = pd.read_excel(file_path)

            # Extract numerical features (Ax, Ay, Az, etc.)
            features = df.iloc[:, 2:-1].values  # Exclude participant_id and last column (Normalized=True)

            # Extract participant labels
            labels = df['participant_id'].values - 1  # Adjust labels to 0-indexed

            # Convert data into frames of 10 timestamps
            for i in range(0, len(features) - 10 + 1, 10):  # Sliding window
                self.data.append(features[i:i+10])
                self.labels.append(labels[i+9])  # Use last timestamp's participant as label

        # Convert lists to NumPy arrays before converting to tensors
        self.data = torch.tensor(np.array(self.data), dtype=torch.float32)
        self.labels = torch.tensor(np.array(self.labels), dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Automatically list all Excel files in the directory
folder_path = "/content/drive/MyDrive/DS"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".xlsx")]

# Create dataset and data loader
dataset = SensorDataset(file_paths)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [5]:
from torch.utils.data import random_split

# Define split sizes
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
import torch.nn as nn
from transformers import BertConfig

class NumericalBERT(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, num_classes, max_seq_length=10):
        super(NumericalBERT, self).__init__()

        self.config = BertConfig(hidden_size=hidden_dim, num_attention_heads=num_heads, num_hidden_layers=num_layers)

        # Numerical embedding layer
        self.embedding = nn.Linear(input_dim, hidden_dim)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_length, hidden_dim))

        # Transformer Encoder
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        # Classification layer
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x + self.positional_encoding[:, :x.size(1), :]
        x = self.encoder(x)
        logits = self.classifier(x[:, -1, :])  # Use last timestamp's representation
        return logits


In [7]:
import torch.nn as nn
import torch.optim as optim

# Model parameters
input_dim = 12  # 12 sensor readings per timestamp
hidden_dim = 128  # Transformer hidden size
num_heads = 4  # Multi-head attention heads
num_layers = 3  # Number of Transformer layers
num_classes = 10  # Participants (1, 2, 3)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NumericalBERT(input_dim, hidden_dim, num_heads, num_layers, num_classes).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [8]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs= 9):
    model.train()  # Set model to training mode

    for epoch in range(num_epochs):
        total_loss = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

        for batch in progress_bar:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Reset gradients
            outputs = model(inputs)  # Forward pass

            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            total_loss += loss.item()

            progress_bar.set_postfix(loss=total_loss / len(train_loader))

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# Train the model
train_model(model, data_loader, criterion, optimizer, num_epochs=9)


Epoch 1/9: 100%|██████████| 63000/63000 [58:19<00:00, 18.00it/s, loss=1.12]


Epoch 1, Loss: 1.1180


Epoch 2/9: 100%|██████████| 63000/63000 [57:23<00:00, 18.29it/s, loss=0.732]


Epoch 2, Loss: 0.7320


Epoch 3/9: 100%|██████████| 63000/63000 [57:04<00:00, 18.40it/s, loss=0.624]


...

Epoch 7, Loss: 0.4828


Epoch 8/9: 100%|██████████| 63000/63000 [57:47<00:00, 18.17it/s, loss=0.466]


Epoch 8, Loss: 0.4662


Epoch 9/9: 100%|██████████| 63000/63000 [58:20<00:00, 18.00it/s, loss=0.453]

Epoch 9, Loss: 0.4526


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1)

            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # F1 Score (macro and per class)
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    print(f"Macro F1 Score: {f1_macro:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Participant 1', 'Participant 2', 'Participant 3', 'Participant 4', 'Participant 5', 'Participant 6', 'Participant 7', 'Participant 8', 'Participant 9', 'Participant 10']))

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10'])

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap="Blues", colorbar=False)
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.tight_layout()
    plt.show()


# Evaluate the model
evaluate_model(model, test_loader)
